In [7]:
import pandas as pd
import numpy as np
import scipy
import qpsolvers
import datetime as dt
from typing import Tuple
import sys
import time
sys.path.insert(1, '../src')

from helper_functions import *
from covariance import Covariance
from optimization import *
from optimization_data import OptimizationData
import qp_problems
import json
from constraints import Constraints

In [8]:
universes = ['msci', 'usa']
solvers = list(set(qpsolvers.solvers.available_solvers) - {'gurobi', 'mosek', 'ecos', 'proxqp', 'piqp', 'scs'}) # FIXME: it depends on the installation?

In [20]:
# Generate constraints
def example_constraints(X):
    constraints = Constraints(selection = X)

    constraints.add_budget()
    constraints.add_box("LongOnly")

    constraints.add_linear(None, pd.Series(np.random.rand(X.size), index=X), '<=', 1)
    constraints.add_linear(None, pd.Series(np.random.rand(X.size), index=X), '>=', -1)
    constraints.add_linear(None, pd.Series(np.random.rand(X.size), index=X), '=', 0.5)

    # sub_universe = X[:X.size // 2]
    # linear_constraints = pd.DataFrame(np.random.rand(3, sub_universe.size), columns=sub_universe)
    # sense = pd.Series(np.repeat('=', 3))
    # rhs = pd.Series(np.ones(3))
    # constraints.add_linear(linear_constraints, None, sense, rhs, None)
    return constraints

def with_l1(X):
    constraints = Constraints(selection = X)

    constraints.add_budget()
    constraints.add_box("LongOnly")

    constraints.add_linear(None, pd.Series(np.random.rand(X.size), index=X), '<=', 1)
    constraints.add_linear(None, pd.Series(np.random.rand(X.size), index=X), '>=', -1)
    constraints.add_linear(None, pd.Series(np.random.rand(X.size), index=X), '=', 0.5)

    linear_constraints = pd.DataFrame(np.random.rand(3, X.size), columns=X)
    sense = pd.Series(np.repeat('<=', 3))
    rhs = pd.Series(np.ones(3))
    constraints.add_linear(linear_constraints, None, sense, rhs, None)
    constraints.add_l1('turnover', rhs = 1, x0 = dict(zip(X, np.zeros(X.size))))
    return constraints

In [10]:
# Initialize optimization object

def prep_optim(optim, constraints):
    # Add constraints
    optim.constraints = constraints

    # Set objective
    optimization_data = OptimizationData(X = data['X'], y = data['y'], align = True)
    optim.set_objective(optimization_data = optimization_data)

    # Ensure that P and q are numpy arrays
    if 'P' in optim.objective.keys():
        if hasattr(optim.objective['P'], "to_numpy"):
            optim.objective['P'] = optim.objective['P'].to_numpy()
    else:
        raise ValueError("Missing matrix 'P' in objective.")
    if 'q' in optim.objective.keys():
        if hasattr(optim.objective['q'], "to_numpy"):
            optim.objective['q'] = optim.objective['q'].to_numpy()
    else:
        optim.objective['q'] = np.zeros(len(optim.constraints.selection))

    # Initialize the optimization model
    optim.model_qpsolvers()
    return optim

In [21]:
def objective_value(optim_objective, w) -> float:
    constant = 0 if optim_objective['constant'] is None else optim_objective['constant'].values
    return 0.5 * (w @ optim_objective['P'] @ w) + optim_objective['q'].T @ w  + constant

# Small dataset msci 

In [22]:
# LOAD DATA
universe = 'msci'
data = load_data(universe)
X = data['X'].columns

In [39]:
msci_ls = {}

constraints = example_constraints(X)
for solver in list(set(qpsolvers.solvers.available_solvers) - {'gurobi', # Restricted license - for non-production use only - expires 2025-11-24
                                                               'mosek', # License cannot be located.
                                                               'ecos', # LinAlgError: 0-dimensional array given. Array must be at least two-dimensional
                                                               'scs', # ValueError: Failed to parse cone field bu
                                                               }):

    init_optim = LeastSquares(solver_name = solver)
    start = time.time()
    optim = prep_optim(init_optim, constraints)
    optim.solve()
    runtime = time.time() - start
    solution = optim.model['solution']
    result = {
              'solution_found' : solution.found,
                'weights' : solution.x,
                'objective_builtin' : solution.obj,
                'primal_residual' :solution.primal_residual(),
                'dual_residual' : solution.dual_residual(),
                'duality_gap' : solution.duality_gap(),
                'runtime' : runtime
                    }
    result['solution'] = solution
    if not(solution.x is None) and (solution.x.any()):
        result['objective_value'] = objective_value(optim.objective, solution.x)[0][0]
        result['max_residual_Ab'] = max(np.abs(optim.model['A'] @ solution.x  - optim.model['b']))
        result['max_residual_Gh'] = max(np.abs(optim.model['G'] @ solution.x  - optim.model['h']))
    else :
        result['objective_value'] = None
    msci_ls[solver] = result


## Comparison of different solvers 

We compare performance of solvers according to their:

* Accuracy: We evaluate objective value at solution found. 

$$\text{objective value} = 0.5*x Px + q^T x$$
* Reliability: We verify whether the sollution satisfies constraints. We compute primal_residual  and dual_residual. These residuals will be zero at the optimum.

$$\text{primal residual} = max(||Ax - b||_{\inf}, [Gx - h]^{+}, [lb - x]^{+}, [x -ub]^{+})$$ 

where $v^{+} = max(v,0)$

$$\text{dual residual} = ||Px + q + A^T y + G^T z + z_{box}  ||$$ 

where 

*    
    *  $y$ : Dual multipliers for equality constraints 

    *   $z$ : Dual multipliers for linear inequality constraints. The dimension of z is equal to the number of inequalities. 
    *   $z_{box}$ : Dual multipliers for box inequality constraints 
    

* Runtime 

In [40]:
pd.DataFrame(msci_ls).drop(index=['solution', 'weights', 'objective_builtin'])

,quadprog,proxqp,highs,clarabel,cvxopt,osqp,piqp,qpalm
solution_found,True,True,True,True,True,True,True,True
primal_residual,0.0,0.000005,0.0,0.0,0.0,0.000242,0.0,0.0
dual_residual,0.0,0.0,0.0,0.0,0.0,0.000816,0.0,0.0
duality_gap,0.0,0.0,0.0,0.0,0.0,0.000592,0.0,0.0
runtime,0.012674,0.012273,0.015811,0.014088,0.0275,0.03881,0.019884,0.027854
objective_value,0.003612,0.003612,0.003612,0.003612,0.003612,0.00362,0.003612,0.003612
max_residual_Ab,0.0,0.0,0.0,0.0,0.0,0.000242,0.0,0.0
max_residual_Gh,1.371239,1.371239,1.371239,1.371239,1.371294,1.371173,1.371239,1.371239


# Large dataset (usa)

In [41]:
# LOAD DATA
universe = 'usa'
data = load_data(universe)
X = data['X'].columns
solver_name = 'cvxopt'

In [42]:
usa_ls = {}
constraints = example_constraints(X)
for solver in list(set(qpsolvers.solvers.available_solvers) - {'gurobi', # Restricted license - for non-production use only - expires 2025-11-24
                                                               'mosek', # License cannot be located.
                                                               'ecos', # LinAlgError: 0-dimensional array given. Array must be at least two-dimensional
                                                               'scs', # ValueError: Failed to parse cone field bu
                                                               }):

    init_optim = LeastSquares(solver_name = solver)
    start = time.time()
    optim = prep_optim(init_optim, constraints)
    optim.solve()
    runtime = time.time() - start
    solution = optim.model['solution']
    result = {
              'solution_found' : solution.found,
                'weights' : solution.x,
                'objective_builtin' : solution.obj,
                'primal_residual' :solution.primal_residual(),
                'dual_residual' : solution.dual_residual(),
                'duality_gap' : solution.duality_gap(),
                'runtime' : runtime
                    }
    result['solution'] = solution
    if not(solution.x is None) and (solution.x.any()):
        result['objective_value'] = objective_value(optim.objective, solution.x)[0][0]
        result['max_residual_Ab'] = max(np.abs(optim.model['A'] @ solution.x  - optim.model['b']))
        result['max_residual_Gh'] = max(np.abs(optim.model['G'] @ solution.x  - optim.model['h']))
    else :
        result['objective_value'] = None
    usa_ls[solver] = result


In [43]:
pd.DataFrame(usa_ls).drop(index=['solution', 'weights', 'objective_builtin'])

,quadprog,proxqp,highs,clarabel,cvxopt,osqp,piqp,qpalm
solution_found,True,True,True,True,True,True,True,True
primal_residual,0.0,0.000008,0.0,0.0,0.0,0.000281,0.0,0.000021
dual_residual,0.0,0.0,0.0,0.0,0.0,0.000994,0.0,0.0
duality_gap,0.0,0.000005,0.0,0.0,0.0,0.000683,0.0,0.000002
runtime,0.512446,0.079288,0.182555,0.473337,0.225714,0.106394,0.092857,0.135028
objective_value,0.008529,0.008524,0.008529,0.008529,0.008529,0.00861,0.008529,0.008528
max_residual_Ab,0.0,0.0,0.0,0.0,0.0,0.000281,0.0,0.0
max_residual_Gh,1.498459,1.498479,1.498459,1.498457,1.498445,1.497347,1.498459,1.498495
